# Simulación de flujo granular con fricción

Este cuaderno ejecuta una simulación simplificada de dinámica molecular (MD) para estudiar el atasco en un silo bidimensional con paredes inclinadas y un orificio en su base. Las partículas son discos blandos que interactúan mediante fuerzas normales y un modelo sencillo de fricción de Coulomb. La puerta del silo permanece cerrada durante una primera fase, permitiendo la compactación del material; transcurrido un tiempo de apertura (`t_open`), el orificio se abre y los discos pueden salir si alcanzan la base en la zona del hueco. Se detiene la simulación cuando no se descarga ningún disco durante un intervalo `stall_time`, interpretándose como atasco.

Las figuras generadas al final reproducen las gráficas del informe: probabilidad de atasco y número medio de discos descargados frente al ancho del orificio (`D`), así como las mismas magnitudes en función del coeficiente de fricción (`mu`) para diversas aberturas. También se explora el efecto de introducir ruido en los radios de las partículas.

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle

# Directorio para guardar figuras
save_dir = './images'

# Parámetros globales
g = -9.81   # gravedad
W = 1.0     # ancho del silo
H = 1.5     # alto del silo
t_open = 0.5  # tiempo hasta abrir el orificio
stall_time = 1.0  # tiempo sin descargas para declarar atasco

# Definición del suelo inclinado: para |x| > D/2 la base sube linealmente con una pendiente

def bottom_height(x, D, slope):
    half = D / 2.0
    return np.where(np.abs(x) <= half, 0.0, slope * (np.abs(x) - half))

# Función de simulación con fricción

def simulate_md(N=30, r=0.02, D=0.1, mu=0.6, slope=1.0, t_open=t_open, stall_time=stall_time, dt=0.001, max_time=2.0, seed=None, noise=False):
    """Simula la caída de N discos blandos en un silo con orificio de ancho D.

    Parámetros:
    - N: número de discos
    - r: radio de los discos (valor medio)
    - D: ancho del orificio
    - mu: coeficiente de fricción de Coulomb (sin dimensión)
    - slope: pendiente de las paredes inclinadas
    - t_open: tiempo tras el cual se abre el orificio
    - stall_time: tiempo sin descargas para considerar atasco
    - dt: paso de integración
    - max_time: tiempo máximo de simulación
    - seed: semilla para reproducibilidad
    - noise: si es True, añade ruido gaussiano al radio de cada disco (std = 0.002)

    Devuelve:
    - jam (bool): True si atasco, False si todos los discos salen
    - removed: número de discos descargados
    """
    rng = np.random.default_rng(seed)
    # Radios con ruido opcional
    if noise:
        radii = np.clip(r + rng.normal(scale=0.002, size=N), 0.005, None)
    else:
        radii = np.full(N, r)
    # Posiciones iniciales: distribuye discos al azar en la parte superior
    positions = np.zeros((N, 2))
    positions[:, 0] = rng.uniform(-W/2 + radii, W/2 - radii)
    positions[:, 1] = H - 2 * radii * rng.random(N)
    # Velocidades iniciales
    velocities = np.zeros((N, 2))
    # Constante de resorte normal
    k = 10000.0
    # Variables de control
    removed_count = 0
    last_removed_time = 0.0
    time = 0.0
    jam = False
    active_mask = np.ones(N, dtype=bool)
    while time < max_time:
        forces = np.zeros((N, 2))
        forces[active_mask, 1] += g
        active_idx = np.where(active_mask)[0]
        # Colisiones entre discos
        for i in range(len(active_idx)):
            a = active_idx[i]
            for j in range(i + 1, len(active_idx)):
                b = active_idx[j]
                delta = positions[b] - positions[a]
                dist = np.linalg.norm(delta)
                overlap = radii[a] + radii[b] - dist
                if overlap > 0:
                    n = delta / (dist + 1e-8)
                    fn = k * overlap
                    dv = velocities[a] - velocities[b]
                    vt = dv - np.dot(dv, n) * n
                    f_t_mag = np.linalg.norm(vt)
                    if f_t_mag > 1e-8:
                        ft_dir = -vt / (f_t_mag + 1e-8)
                        ft = mu * fn * ft_dir
                    else:
                        ft = np.zeros(2)
                    forces[a] -= (fn * n + ft)
                    forces[b] += (fn * n + ft)
        # Colisiones con paredes y suelo inclinado
        for idx in active_idx:
            x, y = positions[idx]
            rad = radii[idx]
            if x - rad < -W/2:
                overlap = -W/2 - (x - rad)
                forces[idx][0] += k * overlap
            if x + rad > W/2:
                overlap = (x + rad) - W/2
                forces[idx][0] -= k * overlap
            bh = bottom_height(x, D, slope)
            if y - rad < bh:
                overlap = bh - (y - rad)
                if abs(x) <= D/2:
                    normal = np.array([0.0, 1.0])
                else:
                    sign = 1.0 if x >= 0 else -1.0
                    slope_vec = np.array([sign, slope])
                    normal = np.array([-slope_vec[1], slope_vec[0]])
                    normal = normal / np.linalg.norm(normal)
                fn = k * overlap
                v_rel = velocities[idx]
                vt = v_rel - np.dot(v_rel, normal) * normal
                f_t_mag = np.linalg.norm(vt)
                if f_t_mag > 1e-8:
                    ft_dir = -vt / (f_t_mag + 1e-8)
                    ft = mu * fn * ft_dir
                else:
                    ft = np.zeros(2)
                forces[idx] += fn * normal + ft
        # Integración
        velocities[active_mask] += forces[active_mask] * dt
        positions[active_mask] += velocities[active_mask] * dt
        time += dt
        # Eliminación de discos tras abrir orificio
        if time > t_open:
            to_remove = []
            for idx in active_idx:
                x, y = positions[idx]
                if y - radii[idx] <= 0 and abs(x) < D/2:
                    to_remove.append(idx)
            if to_remove:
                for idx in to_remove:
                    active_mask[idx] = False
                removed_count += len(to_remove)
                last_removed_time = time
        # Detección de atasco
        if time > t_open and (time - last_removed_time) > stall_time:
            if active_mask.sum() == 0:
                jam = False
            else:
                jam = True
            break
    if time >= max_time and active_mask.sum() > 0:
        jam = True
    return jam, removed_count

# Función de experimentos

def experiment_vs_D(mu=0.6, D_range=np.arange(0.06,0.31,0.03), replicates=3, N=30, r=0.02, noise=False):
    jam_probs = []
    removed_means = []
    for D in D_range:
        jam_count = 0
        removed_list = []
        for rep in range(replicates):
            jam, removed = simulate_md(N=N, r=r, D=D, mu=mu, noise=noise, seed=rep)
            jam_count += int(jam)
            removed_list.append(removed)
        jam_probs.append(jam_count / replicates)
        removed_means.append(np.mean(removed_list))
    return np.array(jam_probs), np.array(removed_means)

# Función en fricción

def experiment_vs_mu(D, mu_values, replicates=3, N=30, r=0.02):
    jam_probs = []
    removed_means = []
    for mu_val in mu_values:
        jam_count = 0
        removed_list = []
        for rep in range(replicates):
            jam, removed = simulate_md(N=N, r=r, D=D, mu=mu_val, seed=rep)
            jam_count += int(jam)
            removed_list.append(removed)
        jam_probs.append(jam_count / replicates)
        removed_means.append(np.mean(removed_list))
    return np.array(jam_probs), np.array(removed_means)

# Experimentos principales
D_values = np.arange(0.06, 0.31, 0.03)
jam_prob_D, removed_D = experiment_vs_D(mu=0.6, D_range=D_values, replicates=2, N=30)

plt.figure(figsize=(5,3))
plt.plot(D_values, jam_prob_D, marker='o')
plt.xlabel('Ancho del orificio D')
plt.ylabel('Probabilidad de atasco')
plt.title('Probabilidad de atasco vs D (mu=0.6)')
plt.grid(True)
plt.savefig(f"{save_dir}/md_sim_jam_vs_D.png", dpi=300, bbox_inches='tight')
plt.close()

plt.figure(figsize=(5,3))
plt.plot(D_values, removed_D, marker='o')
plt.xlabel('Ancho del orificio D')
plt.ylabel('Discos descargados')
plt.title('Discos descargados vs D (mu=0.6)')
plt.grid(True)
plt.savefig(f"{save_dir}/md_sim_removed_vs_D.png", dpi=300, bbox_inches='tight')
plt.close()

selected_D = [0.06, 0.12, 0.18, 0.24, 0.30]
mu_values = [0.0, 0.3, 0.6, 1.0]
jams_vs_mu = {}
removed_vs_mu = {}
for D in selected_D:
    jp, rm = experiment_vs_mu(D, mu_values, replicates=2, N=30)
    jams_vs_mu[D] = jp
    removed_vs_mu[D] = rm

plt.figure(figsize=(5,3))
for D in selected_D:
    plt.plot(mu_values, jams_vs_mu[D], marker='o', label=f'D={D:.2f}')
plt.xlabel('Coeficiente de fricción mu')
plt.ylabel('Probabilidad de atasco')
plt.title('Probabilidad de atasco vs mu para distintos D')
plt.legend()
plt.grid(True)
plt.savefig(f"{save_dir}/md_sim_jam_vs_mu_multiple_D.png", dpi=300, bbox_inches='tight')
plt.close()

plt.figure(figsize=(5,3))
for D in selected_D:
    plt.plot(mu_values, removed_vs_mu[D], marker='o', label=f'D={D:.2f}')
plt.xlabel('Coeficiente de fricción mu')
plt.ylabel('Discos descargados')
plt.title('Discos descargados vs mu para distintos D')
plt.legend()
plt.grid(True)
plt.savefig(f"{save_dir}/md_sim_removed_vs_mu_multiple_D.png", dpi=300, bbox_inches='tight')
plt.close()

# Ruido en radios
jam_noise, removed_noise = experiment_vs_D(mu=0.6, D_range=D_values, replicates=2, N=30, noise=True)
plt.figure(figsize=(5,3))
plt.plot(D_values, jam_noise, marker='o', label='Con ruido')
plt.plot(D_values, jam_prob_D, marker='s', label='Sin ruido')
plt.xlabel('Ancho del orificio D')
plt.ylabel('Probabilidad de atasco')
plt.title('Efecto del ruido en el tamaño de los discos')
plt.legend()
plt.grid(True)
plt.savefig(f"{save_dir}/md_sim_jam_vs_D_noise.png", dpi=300, bbox_inches='tight')
plt.close()

plt.figure(figsize=(5,3))
plt.plot(D_values, removed_noise, marker='o', label='Con ruido')
plt.plot(D_values, removed_D, marker='s', label='Sin ruido')
plt.xlabel('Ancho del orificio D')
plt.ylabel('Discos descargados')
plt.title('Discos descargados vs D con ruido en el tamaño')
plt.legend()
plt.grid(True)
plt.savefig(f"{save_dir}/md_sim_removed_vs_D_noise.png", dpi=300, bbox_inches='tight')
plt.close()

print('Simulaciones completadas. Figuras guardadas.')
